# Gesture Detection Model

## 1 Engineer features

### 1.1 Import libraries and transformers  

In [1]:
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
from keras.utils import to_categorical
warnings.filterwarnings("ignore")

/Users/lsafari/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import os
os.getcwd()

'/Users/lsafari/drone_steering/models/playground'

In [3]:
os.chdir('../..')

In [4]:
from app_local.module import DataEnsembler, LabelGenerator, GestureTransformer

### 1.2  Load data

In [5]:
de = DataEnsembler(ms_per_frame=120)
de.investigate_available_datafiles(data_dir='data/gesture/', is_frame_based = False)
de.combined_data_files_df

,filename_features,filename_labels
0,features_left_c_02_120.csv,labels_left_c_02.csv
1,features_left_l_01_120.csv,labels_left_l_01.csv
2,features_move_p_01_120.csv,labels_move_p_01.csv
3,features_move_c_01_120.csv,labels_move_c_01.csv
4,features_land_l_03_120.csv,labels_land_l_03.csv
5,features_move_l_02_120.csv,labels_move_l_02.csv
6,features_land_c_01_120.csv,labels_land_c_01.csv
7,features_land_l_02_120.csv,labels_land_l_02.csv
8,features_left_c_03_120.csv,labels_left_c_03.csv
9,features_right_c_02_120.csv,labels_right_c_02.csv


In [7]:
de.load_data()
de.assemble_data(max_error=500, tolerance_range= 600)
de.display_information()

i: 0 	shape X: (550, 18, 16) 	shape y: (550,) 	count: 0
i: 1 	shape X: (541, 18, 16) 	shape y: (541,) 	count: 0
i: 2 	shape X: (590, 18, 16) 	shape y: (590,) 	count: 90
i: 3 	shape X: (616, 18, 16) 	shape y: (616,) 	count: 89
i: 4 	shape X: (466, 18, 16) 	shape y: (466,) 	count: 48
i: 5 	shape X: (470, 18, 16) 	shape y: (470,) 	count: 48
i: 6 	shape X: (440, 18, 16) 	shape y: (440,) 	count: 24
i: 7 	shape X: (436, 18, 16) 	shape y: (436,) 	count: 35
i: 8 	shape X: (549, 18, 16) 	shape y: (549,) 	count: 0
i: 9 	shape X: (923, 18, 16) 	shape y: (923,) 	count: 0
i: 10 	shape X: (545, 18, 16) 	shape y: (545,) 	count: 0
i: 11 	shape X: (476, 18, 16) 	shape y: (476,) 	count: 54
i: 12 	shape X: (621, 18, 16) 	shape y: (621,) 	count: 60
i: 13 	shape X: (551, 18, 16) 	shape y: (551,) 	count: 0
i: 14 	shape X: (505, 18, 16) 	shape y: (505,) 	count: 0
i: 15 	shape X: (535, 18, 16) 	shape y: (535,) 	count: 0
i: 16 	shape X: (514, 18, 16) 	shape y: (514,) 	count: 0
i: 17 	shape X: (547, 18, 16) 	sh

### 1.3  Transform data

In [ ]:
from keras.utils import to_categorical

X = de.X.copy()
y = de.y.copy()

print("---------------------------------------------------------------------")
print("Shapes before removal of 'no movements':") 
print("y: " + str(y.shape))
print("X: " + str(X.shape))
print("")

# only select certain indices to prevent too many 0-labeled instances
idx = []
for i in range(7):
    idx.append(np.where(np.isclose(y,i))[0])

print("Labels before removal of 'no movements'")
for i in range(7):
    print(i,len(idx[i]))
print("")
    
zero_idx = np.random.choice(idx[0], 500, replace=False)
keep_idx = np.concatenate([zero_idx,idx[1],idx[2],idx[3],idx[4],idx[5],idx[6]])
keep_idx = sorted(keep_idx)

print("---------------------------------------------------------------------")
y = y[keep_idx]
X = X[keep_idx]
print("Shapes after removal of 'no movements':") 
print("y: " + str(y.shape))
print("X: " + str(X.shape))
print("")

idx = []
for i in range(7):
    idx.append(np.where(np.isclose(y,i))[0])

print("Labels after removal of 'no movements'")
for i in range(7):
    print(i,len(idx[i]))
print("")    
    
# print("---------------------------------------------------------------------")
# print("Features:")
# print(de.LabelGenerators[0].feature_names)


print("---------------------------------------------------------------------")
gt = GestureTransformer(feature_names = list(de.LabelGenerators[0].feature_names),byrow=True)
X = gt.transform(X)

y = to_categorical(y)

print("Shapes after transformation:") 
print("y: " + str(y.shape))
print("X: " + str(X.shape))

## 2 Train model

### 2.1  Split in train / test

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

### 2.2 Define and compile model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM

In [ ]:
n_timesteps = X.shape[1] # here 17
n_features = X.shape[2] # here 16
n_outputs =  y.shape[1] # here 7 (number of labels)

model = Sequential()
model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
model.add(Dropout(0.4))
model.add(Dense(100, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))

model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

model.summary()

### 2.3 Fit and validate model

In [ ]:
history = model.fit(X_train, y_train, epochs=500, batch_size=30, verbose=1, validation_split=0.2)

In [ ]:
r = range(1, len(history.history["acc"]) + 1)

plt.figure(figsize=(20, 5))

plt.subplot(1, 2, 1)
plt.plot(r, history.history["acc"], label="acc")
plt.plot(r, history.history["val_acc"], label="val_acc")
plt.legend()
plt.title("Accuracy")

plt.subplot(1, 2, 2)
plt.plot(r, history.history["loss"], label="loss")
plt.plot(r, history.history["val_loss"], label="val_loss")
plt.legend()
plt.title("Loss")

plt.show()
plt.close()

In [ ]:
eval_results = np.round(model.evaluate(X_test, y_test, batch_size=16, verbose=1),2)
print("Loss:     " + str(eval_results[0]))
print("Accuracy: " + str(eval_results[1]))

In [ ]:
y_test[0]

## ROC analysis

In [ ]:
counter_F_neg=0
counter_F_pos=0
for i in range(X_test.shape[0]):
    if (model.predict_classes(X_test[i:(i+1),:,:])[0]!=np.nonzero(y_test[i])[0][0]):
        if model.predict_classes(X_test[i:(i+1),:,:])[0]==0:
            counter_F_neg=counter_F_neg+1
            print("False_negative i:",i,\
              "\tPred:",model.predict_classes(X_test[i:(i+1),:,:])[0],\
              "\tActual:",np.nonzero(y_test[i])[0][0],\
              "\tProbability:",round(np.max(model.predict_proba(X_test[i:(i+1),:,:])),2),)          
        else:
            counter_F_pos=counter_F_pos+1
            print("False_positive i:",i,\
              "\tPred:",model.predict_classes(X_test[i:(i+1),:,:])[0],\
              "\tActual:",np.nonzero(y_test[i])[0][0],\
              "\tProbability:",round(np.max(model.predict_proba(X_test[i:(i+1),:,:])),2))
print("\ncounter_F_neg:",counter_F_neg) 
print("\ncounter_F_pos:",counter_F_pos) 
         
           

In [ ]:
for i in range(X_test.shape[0]):
    if (model.predict_classes(X_test[i:(i+1),:,:])[0]==np.nonzero(y_test[i])[0][0]):
        print("Index:",i,\
              "\tPred:",model.predict_classes(X_test[i:(i+1),:,:])[0],\
              "\tActual:",np.nonzero(y_test[i])[0][0],\
              "\tProbability:",round(np.max(model.predict_proba(X_test[i:(i+1),:,:])),2))

### 2.4 Save model

In [ ]:
from keras.models import load_model
import h5py

In [ ]:
# save model, delete current reference and re-load it from file
model.save('model_gesture.h5')
del model
model = load_model('model_gesture.h5')

In [ ]:
type(model)